In [1]:
import pandas as pd  
import numpy as np
import re
from unidecode import unidecode 

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep  

In [2]:
def comparador_piensos(nombre_pienso):
    
    """
    Compara los precios de un pienso en varias tiendas en línea.

    Parámetros:
    nombre_pienso (str): El nombre del pienso a comparar.

    Retorna:
    dict: Un diccionario que contiene los precios del pienso en diferentes tiendas.
    """
    
    nombre_pienso = nombre_pienso.lower()

    palabras_pienso = nombre_pienso.split()
    
    tiendas_pienso = ['Amazon', 'Miscota', 'Zooplus', 'Tiendanimal', 'Piensoymascotas', 'Kiwoko']
    
    # Convertir la lista de tiendas en un string separado por comas
    string_tiendas = ', '.join(tiendas_pienso)
    string_tiendas = string_tiendas.rsplit(', ', 1)[0]
    
    resultados = {}
    
    driver = webdriver.Chrome()
    
    # Abrir el navegador y maximizar la ventana
    driver.get("https://www.google.com/")
    driver.maximize_window()
    
    sleep(3)
    
    # Intentar cerrar la ventana emergente si existe
    try:
        driver.find_element("css selector", "#W0wltc").click()
    except:
        pass
    
    print(f'🐶 Obteniendo precios para {nombre_pienso.title()} en: {string_tiendas} 🐕')
    print(f' ')
    
    # Iterar sobre cada tienda para buscar el pienso
    for tienda in tiendas_pienso:
        
        similitudes = {}
        
        # Crear la cadena de búsqueda con el nombre del pienso y el nombre de la tienda
        busqueda = f'{nombre_pienso} {tienda}'
        
        # Esperar 2 segundos antes de realizar la búsqueda
        sleep(2)
        
        # Realizar la búsqueda en Google
        driver.find_element("css selector", "#APjFqb").send_keys(busqueda, Keys.ENTER)
        
        try:
            # Iterar sobre los primeros 5 resultados de búsqueda
            for resultado in range(1, 5):
                
                # Obtener los datos del resultado actual
                datos_resultado = driver.find_element('css selector', f'#rso > div:nth-child({resultado}) > div > div').text.split("\n")
                coincidencia = 0
                
                # Verificar si el nombre de la tienda y una valoración están presentes en el resultado
                if tienda.lower().strip() in datos_resultado[1].lower().strip() and 'valoracion' in unidecode(datos_resultado[-1]).lower().strip():
                    
                    # Contar las palabras del nombre del pienso presentes en el resultado
                    for palabra in palabras_pienso:
                        if palabra in datos_resultado[0].lower().strip():
                            coincidencia += 1
                            
                    # Almacenar la similitud entre la búsqueda y el resultado
                    similitudes[resultado] = coincidencia
                    
            # Encontrar el resultado con la mayor similitud
            tope = max(similitudes.values())
            for res, sim in similitudes.items():
                if sim == tope:
                    resultado_max = res
                    break
                
            # Obtener los datos del resultado con la mayor similitud
            datos_resultado_max = driver.find_element('css selector', f'#rso > div:nth-child({resultado_max}) > div > div').text.split("\n")
            
            try:
                # Extraer el precio del resultado
                try:
                    precio = re.search('hasta (\d+,\d+) €', datos_resultado_max[-1]).group(1)
                    precio = precio.replace(',', '.')
                    precio = float(precio)
                except:
                    precio = re.search('(\d+,\d+) €', datos_resultado_max[-1]).group(1)
                    precio = precio.replace(',', '.')
                    precio = float(precio)
            except:
                precio = np.nan
            
            # Almacenar el precio en el diccionario de resultados
            resultados[tienda] = precio
        
        except:
            # Si hay algún error, asignar NaN al precio de la tienda
            resultados[tienda] = np.nan
        
        sleep(2)
        
        # Hacer clic en el botón de búsqueda para limpiar el campo de búsqueda
        driver.find_element("css selector", "#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef > div").click()
    
    # Encontrar el precio mínimo entre todas las tiendas
    precio_min = np.nanmin(list(resultados.values()))
    
    # Imprimir la tienda con el precio más bajo
    for t, p in resultados.items():
        if p == precio_min:
            print(f'El pienso está más barato en {t}. Sale por {p} €')
            print(' ')
            break
    
    print(f'Puedes consultar el resto de precios en los resultados:')
    return(resultados)

In [3]:
pienso = 'taste of the wild sierra mountain 12,2 kg'

comparador_piensos(pienso)

🐶 Obteniendo precios para Taste Of The Wild Sierra Mountain 12,2 Kg en: Amazon, Miscota, Zooplus, Tiendanimal, Piensoymascotas 🐕
 
El pienso está más barato en Amazon. Sale por 62.49 €
 
Puedes consultar el resto de precios en los resultados:


{'Amazon': 62.49,
 'Miscota': 64.69,
 'Zooplus': 124.99,
 'Tiendanimal': 64.69,
 'Piensoymascotas': 64.99,
 'Kiwoko': 62.49}